In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Importar CSV

In [6]:
# Leer el archivo CSV
df = pd.read_csv('spotify_tracks.csv', sep=',')

# Mostrar las primeras filas del DataFrame
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'spotify_tracks.csv'

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

# Procesamiento de los Datos

In [ ]:
df.drop(columns=["id"],inplace=True)

In [ ]:
df.duplicated(subset=df.drop(columns=["name"]).columns).sum()

In [ ]:
df.drop_duplicates(subset=['genre', 'artists', 'album', 'popularity', 'duration_ms', 'explicit'],inplace=True)

In [ ]:
df.info()

In [ ]:
df["artists"].nunique()

In [ ]:
df["album"].nunique()

In [ ]:
df["duration_ms"].describe()

In [ ]:
df["popularity"].describe()

In [ ]:
df["explicit"].value_counts()

In [ ]:
plt.pie(df["explicit"].value_counts(),labels=df["explicit"].unique(),autopct='%1.2f%%')
plt.show()

# Transformacion de los Datos 

In [ ]:
for column in ["genre", "artists", "album", "explicit"]:
    LE = LabelEncoder()
    df[column] = LE.fit_transform(df[column])

In [ ]:
scaler = StandardScaler()
df[["popularity", "duration_ms"]] = scaler.fit_transform(df[["popularity", "duration_ms"]])

# Crear y Entrenar el Modelo

In [ ]:
similarity_matrix = cosine_similarity(df.drop(columns=["name"]))

In [ ]:
def recommend(song_index, num_recommendations=5):
    similar_songs = list(enumerate(similarity_matrix[song_index]))
    similar_songs = sorted(similar_songs, key=lambda x: x[1], reverse=True)
    similar_songs = similar_songs[1:num_recommendations+1]
    recommendations = [df.iloc[i[0]]['name'] for i in similar_songs]
    return recommendations

In [ ]:
cos_recommendations = recommend(song_index=0, num_recommendations=5)
cos_recommendations

# Crear KNN (Vecinos Cercanos)

In [ ]:
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(df.drop(columns=["name"]))

In [ ]:
distances, indices = knn.kneighbors([df.drop(columns=['name']).iloc[0]], n_neighbors=5)
knn_recommendations = df.iloc[indices[0]]['name'].tolist()

In [ ]:
for idx, rec in enumerate(knn_recommendations, start=1):
    print(f"{idx}. {rec}")

## Autoencoders

In [ ]:
input_dim = df.drop(columns=["name"]).shape[1]

In [ ]:
input_layer = Input(shape=(input_dim,))
encoded = Dense(10, activation='relu')(input_layer) #Encoding the input layer 
decoded = Dense(input_dim, activation='sigmoid')(encoded) #Decoding the encoded layer

In [ ]:
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')